In [1]:
import pandas as pd
import plotly.express as px

In [2]:
clients = pd.read_pickle("data/parsed/clients.pkl")
transactions= pd.read_pickle("data/parsed/transactions.pkl")

Client-Order ranked

In [3]:
idx1 = transactions[["POC","ORDER_ID","INVOICE_DATE"]].drop_duplicates()
idx1 = idx1.sort_values(["POC","INVOICE_DATE"])

idx1["ORDER_RANK"] = idx1.groupby("POC")["INVOICE_DATE"].rank(method="first").astype(int)


Client-Order-Product ranked

In [4]:
idx2 = transactions[["POC","ORDER_ID","INVOICE_DATE","SKU_ID"]].drop_duplicates()
idx2 = idx2.sort_values(["POC","INVOICE_DATE","SKU_ID"])

idx2["ORDER_PRODUCT_RANK"] = idx2.groupby(["POC","SKU_ID"])["INVOICE_DATE"].rank(method="first").astype(int)


Build final DF

In [5]:
all_products = transactions[["SKU_ID"]].drop_duplicates()

transactions_ranked = (all_products
    .merge(idx1, how="cross")
    .merge(idx2, on=["POC","ORDER_ID","INVOICE_DATE","SKU_ID"], how="left")
    .merge(transactions, on=["POC","SKU_ID","ORDER_ID","INVOICE_DATE"], how="left")
    .fillna(0)
    [["POC","SKU_ID","ORDER_ID","INVOICE_DATE","ITEMS_PHYS_CASES","ORDER_RANK","ORDER_PRODUCT_RANK"]]
    .sort_values(["POC","SKU_ID","INVOICE_DATE"])
)

transactions_ranked["ORDER_RANK"] = transactions_ranked["ORDER_RANK"].astype(int)
transactions_ranked["ORDER_PRODUCT_RANK"] = transactions_ranked["ORDER_PRODUCT_RANK"].astype(int)

Stage results

In [6]:
transactions_ranked.to_pickle("data/parsed/transactions_ranked.pkl")